In [1]:
import pandas as pd
from keras.models import load_model
import numpy as np
import sklearn
from sklearn.metrics import roc_curve

k_batch_size = 96

df_test = pd.read_pickle("../../datasets/topoprocessed/test.pkl")
x_test = df_test.loc[:, df_test.columns != 'is_signal_new']
y_test = df_test["is_signal_new"]
del df_test

df_train = pd.read_pickle("../../datasets/topoprocessed/train.pkl")
df_val = pd.read_pickle("../../datasets/topoprocessed/val.pkl")
x_val = df_val.loc[:, df_train.columns != 'is_signal_new']
y_val = df_val["is_signal_new"]
del df_train
del df_val

#Can change to '', '_pt0' or '_pt'
mode = ''
if mode == '_pt0':
    #Get rid of pt_0 column
    x_test = x_test.loc[:, x_test.columns != 'pt_0']
    x_val = x_val.loc[:, x_val.columns != 'pt_0']
elif mode == '_pt':
    pt_cols = [col for col in x_test.columns if 'pt' in col]
    x_test = x_test.drop(pt_cols, axis=1)
    x_val = x_val.drop(pt_cols, axis=1)

model = load_model('topodnnmodels/topodnnmodel' + mode)

2022-06-30 14:57:54.054958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/opt/rh/devtoolset-9/root/usr/lib64/dyninst:/opt/rh/devtoolset-9/root/usr/lib/dyninst:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib
2022-06-30 14:57:54.055611: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-06-30 14:57:54.055967: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Dee

In [2]:
def evaluate_model(model, x_val, y_val):
    score = model.evaluate(x_val, y_val, batch_size=k_batch_size)
    print("Loss: "+str(score[0]))
    print("Acc: "+str(score[1]))
    
    fpr, tpr, thresholds = roc_curve(
            y_val, model.predict(x_val).ravel())
    
    rej_at_30_marker  = None
    rej_at_50_marker  = None
    rej_at_80_marker  = None

    for i in range(len(fpr)):
        if(tpr[i] >= .30 and rej_at_30_marker is None):
            rej_at_30_marker = 1 / fpr[i]
            print("Background rejection at 30\% signal efficiency:" +
                        str(rej_at_30_marker))
        if(tpr[i] >= .50 and rej_at_50_marker is None):
            rej_at_50_marker = 1 / fpr[i]
            print("Background rejection at 50\% signal efficiency:" +
                        str(rej_at_50_marker))
        if(tpr[i] >= .80 and rej_at_80_marker is None):
            rej_at_80_marker = 1 / fpr[i]
            print( "Background rejection at 80\% signal efficiency:" +
                   str(rej_at_80_marker))
    current = sklearn.metrics.auc(fpr, tpr)
    print("AUC:"+str(current))
    return fpr,tpr,current
    
print("Performance on validation set")
_ = evaluate_model(model,x_val,y_val)
print("Performance on test set")
_ = evaluate_model(model,x_test,y_test)

Performance on validation set
4198/4198 [==============================] - 4s 808us/step - loss: 0.2086 - accuracy: 0.9152
Loss: 0.20857317745685577
Acc: 0.9151935577392578
12594/12594 [==============================] - 8s 591us/step
Background rejection at 30\% signal efficiency:250.93773349937732
Background rejection at 50\% signal efficiency:88.22373029772329
Background rejection at 80\% signal efficiency:21.514307068118725
AUC:0.9706369816878555
Performance on test set
4209/4209 [==============================] - 4s 879us/step - loss: 0.2072 - accuracy: 0.9155
Loss: 0.20718255639076233
Acc: 0.9155470132827759
12625/12625 [==============================] - 8s 606us/step
Background rejection at 30\% signal efficiency:278.1184573002755
Background rejection at 50\% signal efficiency:93.4787037037037
Background rejection at 80\% signal efficiency:22.108179130625206
AUC:0.9712025940597585


In [6]:
#Saves prediction values
to_save = model.predict(x_test)
np.save('preds/topodnn' + mode + '.npy',to_save.ravel())